In [ ]:
import numpy as np
import pandas as pd
import os
from topicpy.hsbmpy import get_max_available_L

In [ ]:
directory=f"/home/jovyan/work/phd/datasets/tcga/COAD/mirna_allsamples/"
os.chdir(directory)

In [ ]:
L = get_max_available_L(directory, "trisbm")

In [ ]:
os.listdir()

In [ ]:
df_files = pd.read_csv("files.dat", index_col=0)
df_files.head(2)

In [ ]:
df_isella = pd.read_csv("files_isella.csv", index_col=0)
df_isella.index = ["-".join(sample.split(".")[:3]) for sample in df_isella.index]
df_isella.head(2)

In [ ]:
df_files = df_files.join(df_isella, how="inner")

In [ ]:
df = pd.read_csv("mainTable_all.csv", index_col=0)
df = df.loc[:,df.columns.isin(df_files.index)]
df.head(2)

In [ ]:
directory=f"/home/jovyan/work/phd/datasets/tcga/COAD/isella/"
os.chdir(directory)

In [ ]:
df_files.to_csv("files.dat")

In [ ]:
import graph_tool as gt
import sys
sys.path.append("/home/jovyan/work/phd/trisbm")
from trisbm import trisbm

In [ ]:
model = trisbm()

In [ ]:
model.make_graph(df, lambda gene: 1 if "ENSG" in gene else 2)

In [ ]:
model.save_graph("graph.xml.gz")

## Fit

In [ ]:
model.load_graph("graph.xml.gz")
model.g

In [ ]:
model.fit(n_init=1, B_min=18, B_max=100, overlap=True, deg_corr=True, parallel=True)

In [ ]:
model.save_data()

## Purity

In [ ]:
df_files = pd.read_csv("files.dat")
df_files.head(2)

In [ ]:
df_files["Tumor Purity estimated by Absolute"].hist()

In [ ]:
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

In [ ]:
df_topics = pd.read_csv("trisbm/trisbm_level_1_topic-dist.csv", index_col=1).drop("i_doc", axis=1)
df_topics = df_topics.subtract(df_topics.mean(0),1).divide(df_topics.std(0),1)

In [ ]:
x = df_files["Tumor Purity estimated by Absolute"]

for topic in df_topics.columns:
    y = df_topics.reindex(index=df_files["cases.0.submitter_id"])[topic]
    p = pearsonr(x,y)[0]

    print(p)
    
    plt.scatter(x,y, label=topic)
    
    
plt.xlabel("Tumor purity")
plt.ylabel("P(sample|metadatum)")
plt.legend()
#plt.title("Pearson: {}".format(pearsonr(x,y)[0]))

In [ ]:
df_topics